# BEAT-PD SC3 Subject Specific Models Notebook

In [2]:
# Imports
import os
import sys
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import pytorch_lightning as pl
from pytorch_lightning.logging import TensorBoardLogger
from tensorboard import notebook
from argparse import Namespace
import torch.nn as nn
import torchvision as tv
import torchvision.transforms.functional as TF
from torch.nn import functional as F
from torchvision import transforms, models
from torch.utils import data
from torch.utils.data import DataLoader
from PIL import Image
import random

In [5]:
# Run .py files
%run -i Dataset_ensemble.py
%run -i Torch_Dataset.py
%run -i BEATPD_LSTM.py
%run -i EvalUtilities.py
%run -i LightningEnsemble.py

C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Jonathan\Anaconda3\envs\researchenv\l

plot_ROC and BEATPDscoring commands now loaded.


In [6]:
torch.cuda.empty_cache()

# Identifying Sub Challenge 3 Test Subjects

In [7]:
cis_tst_id = pd.read_csv('Data/BEATPDchallenge/CIS-PD_Test_Data_IDs.csv')

In [8]:
real_tst_id = pd.read_csv('Data/BEATPDchallenge/REAL-PD_Test_Data_IDs.csv')

In [9]:
tst_sc3 = pd.read_csv('Data/BEATPDchallenge/BEAT-PD_SC3_Tremor_Submission_Template.csv')

In [10]:
tst_sc3['subject_id'] = np.nan

In [11]:
for idx1, row1 in tst_sc3.iterrows():
    for idx2, row2 in pd.concat([cis_tst_id,real_tst_id]).reset_index(drop=True).iterrows():
        if row1.measurement_id == row2.measurement_id:
            tst_sc3.subject_id[idx1] = row2.subject_id

LightningEnsemble.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import pytorch_lightning as pl
C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [12]:
 tst_sc3.subject_id.unique()

array([1004.0, 1006.0, 1007.0, 1019.0, 1020.0, 1023.0, 1032.0, 1034.0,
       1038.0, 1043.0, 1046.0, 1048.0, 1049.0, 'hbv013', 'hbv038',
       'hbv023', 'hbv054', 'hbv022', 'hbv012'], dtype=object)

# Create CIS Dataset

In [13]:
############### CIS config ###############

dataset_name = 'CIS'
label_class = 'tremor'
sort_by = 'subject_id'
train_ts_dir = 'Data/BEATPDchallenge/training_data-cispd/'
train_label_dir = 'Data/BEATPDchallenge/data_labels-cispd/CIS-PD_Training_Data_IDs_Labels.csv'
ancil_ts_dir = 'Data/BEATPDchallenge/ancillary_data-cispd/'
ancil_label_dir = 'Data/BEATPDchallenge/data_labels-cispd/CIS-PD_Ancillary_Data_IDs_Labels.csv'
test_ts_dir='Data/BEATPDchallenge/testing_data-cispd/'
test_data_id_dir='Data/BEATPDchallenge/CIS-PD_Test_Data_IDs.csv'

In [14]:
dataset = Dataset(dataset_name=dataset_name, sort_by=sort_by, label_class = label_class,
                           train_ts_dir=train_ts_dir, train_label_dir=train_label_dir,
                           ancil_ts_dir=ancil_ts_dir, ancil_label_dir=ancil_label_dir,
                           combine_ancil=False,test_ts_dir=test_ts_dir, test_data_id_dir=test_data_id_dir)

Creating Test Dictionary: 100%|██████████████████████████████████████████████████████| 618/618 [00:54<00:00, 11.38it/s]


In [15]:
dataset.data_dict.keys()

dict_keys([1004, 1006, 1007, 1019, 1020, 1023, 1032, 1034, 1038, 1039, 1043, 1044, 1046, 1048, 1049, 1051])

In [16]:
dataset.test_data_dict.keys()

dict_keys([1004, 1006, 1007, 1019, 1020, 1023, 1032, 1034, 1038, 1039, 1043, 1044, 1046, 1048, 1049, 1051])

## Model's Settings

In [17]:
model_name = "Final Ensemble 1"
path_to_null_preds = 'Data/BEATPDchallenge/null_preds.csv'
checkpoint_callback = False
early_stop_callback=True 
max_epochs=15
gpus=1 # Set to None for CPU training (**will have to remove .cuda() calls within notebook**)
learning_rate = 0.001
train_batch_size = 5
val_batch_size = 5

# Subject 1004 Model

In [18]:
subject_id = 1004

In [19]:
dataset.run_preprocessing(specific_key_dataset=subject_id, create_synthetic_samples=True, k_neighbors=5)

stage II Preprocessing: 100%|████████████████████████████████████████████████████████████| 1/1 [00:05<00:00,  5.27s/it]


In [20]:
dataset.run_test_preprocessing(specific_key_dataset=subject_id)

Preprocessing Test Data: 100%|███████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.13it/s]


## Train Validation Split

In [21]:
dataset.train_validation_split(val_proportion=0.2)

In [22]:
print('Training size:',len(dataset.train_list))
print('Validation size:',len(dataset.val_list))

Training size: 164
Validation size: 40


## Run Training/Validation

In [23]:
# Hyperparameters
hparams = Namespace(**{'learning_rate':learning_rate, 'train_batch_size': train_batch_size,
                       'val_batch_size': val_batch_size})

In [3]:
# Start tensorboard
notebook.start(f'--logdir lightning_logs --host localhost --port 4999')

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 7532.

In [4]:
notebook.display(port=4999)

Selecting TensorBoard with logdir lightning_logs (started 0:00:01 ago; port 4999, pid 14432).


In [24]:
# Run training/validation of model
model = LightningEnsemble(dataset_name=dataset_name,hparams=hparams,train_list=dataset.train_list,
                       val_list=dataset.val_list,label_class=label_class,
                          path_to_null_preds=path_to_null_preds, gpus=gpus)
logger = TensorBoardLogger("lightning_logs", name=model_name, version=f'subject_{subject_id}_{label_class}')
trainer = pl.Trainer(early_stop_callback=early_stop_callback,
                     max_epochs=max_epochs, gpus=gpus, logger=logger,checkpoint_callback=False)
trainer.fit(model)

C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\gradient_accumulation_scheduler.py:43: DeprecationWarning: Epochs indexing of `scheduling` starts from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\trainer\training_loop.py:280: DeprecationWarning: Displayed epoch numbers in the progress bar start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\early_stopping.py:117: DeprecationWarning: Displayed epoch numbers by `EarlyStopping` start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


1

In [25]:
# List tensorboard instances
#notebook.list()

In [26]:
#tensorboard kill
#!taskkill /pid 7496 /f

In [27]:
#torch.cuda.empty_cache()

## Make Predictions

In [28]:
# Predict and fill in submission template
for spec, id_, subject, ts in dataset.test_data_list:
    output = model([torch.unsqueeze(TF.to_tensor(spec).cuda(), dim=0),
                    torch.unsqueeze(torch.tensor(ts).type('torch.FloatTensor').view(ts.shape[0], -1),dim=0),
                   subject])
    prediction = output.argmax(dim=1, keepdim=True).item()
    df_index = tst_sc3.index[tst_sc3['measurement_id'] == id_]
    tst_sc3.prediction[df_index] = prediction

LightningEnsemble.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import torch.nn as nn


In [29]:
# View predictions
tst_sc3.head()

,measurement_id,prediction,subject_id
0,0e80673b-8c90-474f-8388-00fdf25795e4,4.0,1004
1,a7b61d40-9f26-4878-a2d7-bd766b8ae6f5,2.0,1004
2,fc851afd-133c-4e8f-8deb-0079cc442358,2.0,1004
3,90c2c320-1007-47b4-9f8e-cdd37067dfb2,2.0,1004
4,379fd23b-e38f-4214-8a70-7a829004359f,0.0,1004


In [30]:
 tst_sc3.subject_id.unique()

array([1004.0, 1006.0, 1007.0, 1019.0, 1020.0, 1023.0, 1032.0, 1034.0,
       1038.0, 1043.0, 1046.0, 1048.0, 1049.0, 'hbv013', 'hbv038',
       'hbv023', 'hbv054', 'hbv022', 'hbv012'], dtype=object)

# Subject 1006 Model

In [31]:
torch.cuda.empty_cache()

In [32]:
subject_id = 1006

In [33]:
dataset.run_preprocessing(specific_key_dataset=subject_id, create_synthetic_samples=True, k_neighbors=5)

stage II Preprocessing: 100%|████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.12s/it]


In [34]:
dataset.run_test_preprocessing(specific_key_dataset=subject_id)

Preprocessing Test Data: 100%|███████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.03it/s]


## Train Validation Split

In [35]:
dataset.train_validation_split(val_proportion=0.2)

In [36]:
print('Training size:',len(dataset.train_list))
print('Validation size:',len(dataset.val_list))

Training size: 44
Validation size: 10


## Run Training/Validation

In [37]:
# Hyperparameters
hparams = Namespace(**{'learning_rate':learning_rate, 'train_batch_size': train_batch_size,
                       'val_batch_size': val_batch_size})

In [38]:
# Run training/validation of model
model = LightningEnsemble(dataset_name=dataset_name,hparams=hparams,train_list=dataset.train_list,
                       val_list=dataset.val_list,label_class=label_class,
                          path_to_null_preds=path_to_null_preds, gpus=gpus)
logger = TensorBoardLogger("lightning_logs", name=model_name, version=f'subject_{subject_id}_{label_class}')
trainer = pl.Trainer(early_stop_callback=early_stop_callback,
                     max_epochs=max_epochs, gpus=gpus, logger=logger,checkpoint_callback=False)
trainer.fit(model)

C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\gradient_accumulation_scheduler.py:43: DeprecationWarning: Epochs indexing of `scheduling` starts from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\trainer\training_loop.py:280: DeprecationWarning: Displayed epoch numbers in the progress bar start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


1

## Make Predictions

In [39]:
# Predict and fill in submission template
for spec, id_, subject, ts in dataset.test_data_list:
    output = model([torch.unsqueeze(TF.to_tensor(spec).cuda(), dim=0),
                    torch.unsqueeze(torch.tensor(ts).type('torch.FloatTensor').view(ts.shape[0], -1),dim=0),
                   subject])
    prediction = output.argmax(dim=1, keepdim=True).item()
    df_index = tst_sc3.index[tst_sc3['measurement_id'] == id_]
    tst_sc3.prediction[df_index] = prediction

LightningEnsemble.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import torch.nn as nn


In [40]:
 tst_sc3.subject_id.unique()

array([1004.0, 1006.0, 1007.0, 1019.0, 1020.0, 1023.0, 1032.0, 1034.0,
       1038.0, 1043.0, 1046.0, 1048.0, 1049.0, 'hbv013', 'hbv038',
       'hbv023', 'hbv054', 'hbv022', 'hbv012'], dtype=object)

# Subject 1007 Model

In [41]:
torch.cuda.empty_cache()

In [42]:
subject_id = 1007

In [43]:
dataset.run_preprocessing(specific_key_dataset=subject_id, create_synthetic_samples=True, k_neighbors=5)

stage II Preprocessing: 100%|████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.62s/it]


In [44]:
dataset.run_test_preprocessing(specific_key_dataset=subject_id)

Preprocessing Test Data: 100%|███████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.60s/it]


## Train Validation Split

In [45]:
dataset.train_validation_split(val_proportion=0.2)

In [46]:
print('Training size:',len(dataset.train_list))
print('Validation size:',len(dataset.val_list))

Training size: 240
Validation size: 59


## Run Training/Validation

In [47]:
# Hyperparameters
hparams = Namespace(**{'learning_rate':learning_rate, 'train_batch_size': train_batch_size,
                       'val_batch_size': val_batch_size})

In [48]:
# Run training/validation of model
model = LightningEnsemble(dataset_name=dataset_name,hparams=hparams,train_list=dataset.train_list,
                       val_list=dataset.val_list,label_class=label_class,
                          path_to_null_preds=path_to_null_preds, gpus=gpus)
logger = TensorBoardLogger("lightning_logs", name=model_name, version=f'subject_{subject_id}_{label_class}')
trainer = pl.Trainer(early_stop_callback=early_stop_callback,
                     max_epochs=max_epochs, gpus=gpus, logger=logger,checkpoint_callback=False)
trainer.fit(model)

C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\gradient_accumulation_scheduler.py:43: DeprecationWarning: Epochs indexing of `scheduling` starts from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\trainer\training_loop.py:280: DeprecationWarning: Displayed epoch numbers in the progress bar start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\early_stopping.py:117: DeprecationWarning: Displayed epoch numbers by `EarlyStopping` start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


1

## Make Predictions

In [49]:
# Predict and fill in submission template
for spec, id_, subject, ts in dataset.test_data_list:
    output = model([torch.unsqueeze(TF.to_tensor(spec).cuda(), dim=0),
                    torch.unsqueeze(torch.tensor(ts).type('torch.FloatTensor').view(ts.shape[0], -1),dim=0),
                   subject])
    prediction = output.argmax(dim=1, keepdim=True).item()
    df_index = tst_sc3.index[tst_sc3['measurement_id'] == id_]
    tst_sc3.prediction[df_index] = prediction

LightningEnsemble.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import torch.nn as nn


In [50]:
 tst_sc3.subject_id.unique()

array([1004.0, 1006.0, 1007.0, 1019.0, 1020.0, 1023.0, 1032.0, 1034.0,
       1038.0, 1043.0, 1046.0, 1048.0, 1049.0, 'hbv013', 'hbv038',
       'hbv023', 'hbv054', 'hbv022', 'hbv012'], dtype=object)

# Subject 1019 Model

In [51]:
torch.cuda.empty_cache()

In [52]:
subject_id = 1019

In [53]:
dataset.run_preprocessing(specific_key_dataset=subject_id, create_synthetic_samples=True, k_neighbors=5)

stage II Preprocessing: 100%|████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.52s/it]


In [54]:
dataset.run_test_preprocessing(specific_key_dataset=subject_id)

Preprocessing Test Data: 100%|███████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.91it/s]


## Train Validation Split

In [55]:
dataset.train_validation_split(val_proportion=0.2)

In [56]:
print('Training size:',len(dataset.train_list))
print('Validation size:',len(dataset.val_list))

Training size: 58
Validation size: 14


## Run Training/Validation

In [57]:
# Hyperparameters
hparams = Namespace(**{'learning_rate':learning_rate, 'train_batch_size': train_batch_size,
                       'val_batch_size': val_batch_size})

In [58]:
# Run training/validation of model
model = LightningEnsemble(dataset_name=dataset_name,hparams=hparams,train_list=dataset.train_list,
                       val_list=dataset.val_list,label_class=label_class,
                          path_to_null_preds=path_to_null_preds, gpus=gpus)
logger = TensorBoardLogger("lightning_logs", name=model_name, version=f'subject_{subject_id}_{label_class}')
trainer = pl.Trainer(early_stop_callback=early_stop_callback,
                     max_epochs=max_epochs, gpus=gpus, logger=logger,checkpoint_callback=False)
trainer.fit(model)

C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\gradient_accumulation_scheduler.py:43: DeprecationWarning: Epochs indexing of `scheduling` starts from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\trainer\training_loop.py:280: DeprecationWarning: Displayed epoch numbers in the progress bar start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\early_stopping.py:117: DeprecationWarning: Displayed epoch numbers by `EarlyStopping` start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


1

## Make Predictions

In [59]:
# Predict and fill in submission template
for spec, id_, subject, ts in dataset.test_data_list:
    output = model([torch.unsqueeze(TF.to_tensor(spec).cuda(), dim=0),
                    torch.unsqueeze(torch.tensor(ts).type('torch.FloatTensor').view(ts.shape[0], -1),dim=0),
                   subject])
    prediction = output.argmax(dim=1, keepdim=True).item()
    df_index = tst_sc3.index[tst_sc3['measurement_id'] == id_]
    tst_sc3.prediction[df_index] = prediction

LightningEnsemble.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import torch.nn as nn


In [60]:
 tst_sc3.subject_id.unique()

array([1004.0, 1006.0, 1007.0, 1019.0, 1020.0, 1023.0, 1032.0, 1034.0,
       1038.0, 1043.0, 1046.0, 1048.0, 1049.0, 'hbv013', 'hbv038',
       'hbv023', 'hbv054', 'hbv022', 'hbv012'], dtype=object)

# Subject 1020 Model

In [61]:
torch.cuda.empty_cache()

In [62]:
subject_id = 1020

In [63]:
dataset.run_preprocessing(specific_key_dataset=subject_id, create_synthetic_samples=True, k_neighbors=5)

stage II Preprocessing: 100%|████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.40s/it]


In [64]:
dataset.run_test_preprocessing(specific_key_dataset=subject_id)

Preprocessing Test Data: 100%|███████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.23s/it]


## Train Validation Split

In [65]:
dataset.train_validation_split(val_proportion=0.2)

In [66]:
print('Training size:',len(dataset.train_list))
print('Validation size:',len(dataset.val_list))

Training size: 156
Validation size: 39


## Run Training/Validation

In [67]:
# Hyperparameters
hparams = Namespace(**{'learning_rate':learning_rate, 'train_batch_size': train_batch_size,
                       'val_batch_size': val_batch_size})

In [68]:
# Run training/validation of model
model = LightningEnsemble(dataset_name=dataset_name,hparams=hparams,train_list=dataset.train_list,
                       val_list=dataset.val_list,label_class=label_class,
                          path_to_null_preds=path_to_null_preds, gpus=gpus)
logger = TensorBoardLogger("lightning_logs", name=model_name, version=f'subject_{subject_id}_{label_class}')
trainer = pl.Trainer(early_stop_callback=early_stop_callback,
                     max_epochs=max_epochs, gpus=gpus, logger=logger,checkpoint_callback=False)
trainer.fit(model)

C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\gradient_accumulation_scheduler.py:43: DeprecationWarning: Epochs indexing of `scheduling` starts from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\trainer\training_loop.py:280: DeprecationWarning: Displayed epoch numbers in the progress bar start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\early_stopping.py:117: DeprecationWarning: Displayed epoch numbers by `EarlyStopping` start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


1

## Make Predictions

In [69]:
# Predict and fill in submission template
for spec, id_, subject, ts in dataset.test_data_list:
    output = model([torch.unsqueeze(TF.to_tensor(spec).cuda(), dim=0),
                    torch.unsqueeze(torch.tensor(ts).type('torch.FloatTensor').view(ts.shape[0], -1),dim=0),
                   subject])
    prediction = output.argmax(dim=1, keepdim=True).item()
    df_index = tst_sc3.index[tst_sc3['measurement_id'] == id_]
    tst_sc3.prediction[df_index] = prediction

LightningEnsemble.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import torch.nn as nn


In [70]:
 tst_sc3.subject_id.unique()

array([1004.0, 1006.0, 1007.0, 1019.0, 1020.0, 1023.0, 1032.0, 1034.0,
       1038.0, 1043.0, 1046.0, 1048.0, 1049.0, 'hbv013', 'hbv038',
       'hbv023', 'hbv054', 'hbv022', 'hbv012'], dtype=object)

# Subject 1023 Model

In [71]:
torch.cuda.empty_cache()

In [72]:
subject_id = 1023

In [73]:
dataset.run_preprocessing(specific_key_dataset=subject_id, create_synthetic_samples=True, k_neighbors=5)

stage II Preprocessing: 100%|████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.73s/it]


In [74]:
dataset.run_test_preprocessing(specific_key_dataset=subject_id)

Preprocessing Test Data: 100%|███████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.22s/it]


## Train Validation Split

In [75]:
dataset.train_validation_split(val_proportion=0.2)

In [76]:
print('Training size:',len(dataset.train_list))
print('Validation size:',len(dataset.val_list))

Training size: 236
Validation size: 59


## Run Training/Validation

In [77]:
# Hyperparameters
hparams = Namespace(**{'learning_rate':learning_rate, 'train_batch_size': train_batch_size,
                       'val_batch_size': val_batch_size})

In [78]:
# Run training/validation of model
model = LightningEnsemble(dataset_name=dataset_name,hparams=hparams,train_list=dataset.train_list,
                       val_list=dataset.val_list,label_class=label_class,
                          path_to_null_preds=path_to_null_preds, gpus=gpus)
logger = TensorBoardLogger("lightning_logs", name=model_name, version=f'subject_{subject_id}_{label_class}')
trainer = pl.Trainer(early_stop_callback=early_stop_callback,
                     max_epochs=max_epochs, gpus=gpus, logger=logger,checkpoint_callback=False)
trainer.fit(model)

C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\gradient_accumulation_scheduler.py:43: DeprecationWarning: Epochs indexing of `scheduling` starts from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\trainer\training_loop.py:280: DeprecationWarning: Displayed epoch numbers in the progress bar start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


1

## Make Predictions

In [79]:
# Predict and fill in submission template
for spec, id_, subject, ts in dataset.test_data_list:
    output = model([torch.unsqueeze(TF.to_tensor(spec).cuda(), dim=0),
                    torch.unsqueeze(torch.tensor(ts).type('torch.FloatTensor').view(ts.shape[0], -1),dim=0),
                   subject])
    prediction = output.argmax(dim=1, keepdim=True).item()
    df_index = tst_sc3.index[tst_sc3['measurement_id'] == id_]
    tst_sc3.prediction[df_index] = prediction

LightningEnsemble.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import torch.nn as nn


In [80]:
 tst_sc3.subject_id.unique()

array([1004.0, 1006.0, 1007.0, 1019.0, 1020.0, 1023.0, 1032.0, 1034.0,
       1038.0, 1043.0, 1046.0, 1048.0, 1049.0, 'hbv013', 'hbv038',
       'hbv023', 'hbv054', 'hbv022', 'hbv012'], dtype=object)

# Subject 1032 Model

In [81]:
torch.cuda.empty_cache()

In [82]:
subject_id = 1032

In [83]:
dataset.run_preprocessing(specific_key_dataset=subject_id, create_synthetic_samples=True, k_neighbors=5)

stage II Preprocessing: 100%|████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.80s/it]


In [84]:
dataset.run_test_preprocessing(specific_key_dataset=subject_id)

Preprocessing Test Data: 100%|███████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.02s/it]


## Train Validation Split

In [85]:
dataset.train_validation_split(val_proportion=0.2)

In [86]:
print('Training size:',len(dataset.train_list))
print('Validation size:',len(dataset.val_list))

Training size: 142
Validation size: 35


## Run Training/Validation

In [87]:
# Hyperparameters
hparams = Namespace(**{'learning_rate':learning_rate, 'train_batch_size': train_batch_size,
                       'val_batch_size': val_batch_size})

In [88]:
# Run training/validation of model
model = LightningEnsemble(dataset_name=dataset_name,hparams=hparams,train_list=dataset.train_list,
                       val_list=dataset.val_list,label_class=label_class,
                          path_to_null_preds=path_to_null_preds, gpus=gpus)
logger = TensorBoardLogger("lightning_logs", name=model_name, version=f'subject_{subject_id}_{label_class}')
trainer = pl.Trainer(early_stop_callback=early_stop_callback,
                     max_epochs=max_epochs, gpus=gpus, logger=logger,checkpoint_callback=False)
trainer.fit(model)

C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\gradient_accumulation_scheduler.py:43: DeprecationWarning: Epochs indexing of `scheduling` starts from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\trainer\training_loop.py:280: DeprecationWarning: Displayed epoch numbers in the progress bar start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\early_stopping.py:117: DeprecationWarning: Displayed epoch numbers by `EarlyStopping` start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


1

## Make Predictions

In [89]:
# Predict and fill in submission template
for spec, id_, subject, ts in dataset.test_data_list:
    output = model([torch.unsqueeze(TF.to_tensor(spec).cuda(), dim=0),
                    torch.unsqueeze(torch.tensor(ts).type('torch.FloatTensor').view(ts.shape[0], -1),dim=0),
                   subject])
    prediction = output.argmax(dim=1, keepdim=True).item()
    df_index = tst_sc3.index[tst_sc3['measurement_id'] == id_]
    tst_sc3.prediction[df_index] = prediction

LightningEnsemble.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import torch.nn as nn


In [90]:
 tst_sc3.subject_id.unique()

array([1004.0, 1006.0, 1007.0, 1019.0, 1020.0, 1023.0, 1032.0, 1034.0,
       1038.0, 1043.0, 1046.0, 1048.0, 1049.0, 'hbv013', 'hbv038',
       'hbv023', 'hbv054', 'hbv022', 'hbv012'], dtype=object)

# Subject 1034 Model

In [91]:
torch.cuda.empty_cache()

In [92]:
subject_id = 1034

In [93]:
dataset.run_preprocessing(specific_key_dataset=subject_id, create_synthetic_samples=True, k_neighbors=5)

stage II Preprocessing: 100%|████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.16s/it]


In [94]:
dataset.run_test_preprocessing(specific_key_dataset=subject_id)

Preprocessing Test Data: 100%|███████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.76it/s]


## Train Validation Split

In [95]:
dataset.train_validation_split(val_proportion=0.2)

In [96]:
print('Training size:',len(dataset.train_list))
print('Validation size:',len(dataset.val_list))

Training size: 48
Validation size: 11


## Run Training/Validation

In [97]:
# Hyperparameters
hparams = Namespace(**{'learning_rate':learning_rate, 'train_batch_size': train_batch_size,
                       'val_batch_size': val_batch_size})

In [98]:
# Run training/validation of model
model = LightningEnsemble(dataset_name=dataset_name,hparams=hparams,train_list=dataset.train_list,
                       val_list=dataset.val_list,label_class=label_class,
                          path_to_null_preds=path_to_null_preds, gpus=gpus)
logger = TensorBoardLogger("lightning_logs", name=model_name, version=f'subject_{subject_id}_{label_class}')
trainer = pl.Trainer(early_stop_callback=early_stop_callback,
                     max_epochs=max_epochs, gpus=gpus, logger=logger,checkpoint_callback=False)
trainer.fit(model)

C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\gradient_accumulation_scheduler.py:43: DeprecationWarning: Epochs indexing of `scheduling` starts from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\trainer\training_loop.py:280: DeprecationWarning: Displayed epoch numbers in the progress bar start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


1

## Make Predictions

In [99]:
# Predict and fill in submission template
for spec, id_, subject, ts in dataset.test_data_list:
    output = model([torch.unsqueeze(TF.to_tensor(spec).cuda(), dim=0),
                    torch.unsqueeze(torch.tensor(ts).type('torch.FloatTensor').view(ts.shape[0], -1),dim=0),
                   subject])
    prediction = output.argmax(dim=1, keepdim=True).item()
    df_index = tst_sc3.index[tst_sc3['measurement_id'] == id_]
    tst_sc3.prediction[df_index] = prediction

LightningEnsemble.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import torch.nn as nn


In [100]:
 tst_sc3.subject_id.unique()

array([1004.0, 1006.0, 1007.0, 1019.0, 1020.0, 1023.0, 1032.0, 1034.0,
       1038.0, 1043.0, 1046.0, 1048.0, 1049.0, 'hbv013', 'hbv038',
       'hbv023', 'hbv054', 'hbv022', 'hbv012'], dtype=object)

# Subject 1038 Model

In [101]:
torch.cuda.empty_cache()

In [102]:
subject_id = 1038

In [103]:
dataset.run_preprocessing(specific_key_dataset=subject_id, create_synthetic_samples=True, k_neighbors=5)

stage II Preprocessing: 100%|████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.73s/it]


In [104]:
dataset.run_test_preprocessing(specific_key_dataset=subject_id)

Preprocessing Test Data: 100%|███████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.52s/it]


## Train Validation Split

In [105]:
dataset.train_validation_split(val_proportion=0.2)

In [106]:
print('Training size:',len(dataset.train_list))
print('Validation size:',len(dataset.val_list))

Training size: 166
Validation size: 41


## Run Training/Validation

In [107]:
# Hyperparameters
hparams = Namespace(**{'learning_rate':learning_rate, 'train_batch_size': train_batch_size,
                       'val_batch_size': val_batch_size})

In [108]:
# Run training/validation of model
model = LightningEnsemble(dataset_name=dataset_name,hparams=hparams,train_list=dataset.train_list,
                       val_list=dataset.val_list,label_class=label_class,
                          path_to_null_preds=path_to_null_preds, gpus=gpus)
logger = TensorBoardLogger("lightning_logs", name=model_name, version=f'subject_{subject_id}_{label_class}')
trainer = pl.Trainer(early_stop_callback=early_stop_callback,
                     max_epochs=max_epochs, gpus=gpus, logger=logger,checkpoint_callback=False)
trainer.fit(model)

C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\gradient_accumulation_scheduler.py:43: DeprecationWarning: Epochs indexing of `scheduling` starts from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\trainer\training_loop.py:280: DeprecationWarning: Displayed epoch numbers in the progress bar start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\early_stopping.py:117: DeprecationWarning: Displayed epoch numbers by `EarlyStopping` start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


1

## Make Predictions

In [109]:
# Predict and fill in submission template
for spec, id_, subject, ts in dataset.test_data_list:
    output = model([torch.unsqueeze(TF.to_tensor(spec).cuda(), dim=0),
                    torch.unsqueeze(torch.tensor(ts).type('torch.FloatTensor').view(ts.shape[0], -1),dim=0),
                   subject])
    prediction = output.argmax(dim=1, keepdim=True).item()
    df_index = tst_sc3.index[tst_sc3['measurement_id'] == id_]
    tst_sc3.prediction[df_index] = prediction

LightningEnsemble.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import torch.nn as nn


In [110]:
 tst_sc3.subject_id.unique()

array([1004.0, 1006.0, 1007.0, 1019.0, 1020.0, 1023.0, 1032.0, 1034.0,
       1038.0, 1043.0, 1046.0, 1048.0, 1049.0, 'hbv013', 'hbv038',
       'hbv023', 'hbv054', 'hbv022', 'hbv012'], dtype=object)

# Subject 1043 Model

In [111]:
torch.cuda.empty_cache()

In [112]:
subject_id = 1043

In [113]:
dataset.run_preprocessing(specific_key_dataset=subject_id, create_synthetic_samples=True, k_neighbors=5)

stage II Preprocessing: 100%|████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.07it/s]


In [114]:
dataset.run_test_preprocessing(specific_key_dataset=subject_id)

Preprocessing Test Data: 100%|███████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.40it/s]


## Train Validation Split

In [115]:
dataset.train_validation_split(val_proportion=0.2)

In [116]:
print('Training size:',len(dataset.train_list))
print('Validation size:',len(dataset.val_list))

Training size: 39
Validation size: 9


## Run Training/Validation

In [117]:
# Hyperparameters
hparams = Namespace(**{'learning_rate':learning_rate, 'train_batch_size': train_batch_size,
                       'val_batch_size': val_batch_size})

In [118]:
# Run training/validation of model
model = LightningEnsemble(dataset_name=dataset_name,hparams=hparams,train_list=dataset.train_list,
                       val_list=dataset.val_list,label_class=label_class,
                          path_to_null_preds=path_to_null_preds, gpus=gpus)
logger = TensorBoardLogger("lightning_logs", name=model_name, version=f'subject_{subject_id}_{label_class}')
trainer = pl.Trainer(early_stop_callback=early_stop_callback,
                     max_epochs=max_epochs, gpus=gpus, logger=logger,checkpoint_callback=False)
trainer.fit(model)

C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\gradient_accumulation_scheduler.py:43: DeprecationWarning: Epochs indexing of `scheduling` starts from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\trainer\training_loop.py:280: DeprecationWarning: Displayed epoch numbers in the progress bar start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\early_stopping.py:117: DeprecationWarning: Displayed epoch numbers by `EarlyStopping` start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


1

## Make Predictions

In [119]:
# Predict and fill in submission template
for spec, id_, subject, ts in dataset.test_data_list:
    output = model([torch.unsqueeze(TF.to_tensor(spec).cuda(), dim=0),
                    torch.unsqueeze(torch.tensor(ts).type('torch.FloatTensor').view(ts.shape[0], -1),dim=0),
                   subject])
    prediction = output.argmax(dim=1, keepdim=True).item()
    df_index = tst_sc3.index[tst_sc3['measurement_id'] == id_]
    tst_sc3.prediction[df_index] = prediction

LightningEnsemble.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import torch.nn as nn


In [120]:
 tst_sc3.subject_id.unique()

array([1004.0, 1006.0, 1007.0, 1019.0, 1020.0, 1023.0, 1032.0, 1034.0,
       1038.0, 1043.0, 1046.0, 1048.0, 1049.0, 'hbv013', 'hbv038',
       'hbv023', 'hbv054', 'hbv022', 'hbv012'], dtype=object)

# Subject 1046 Model

In [121]:
torch.cuda.empty_cache()

In [122]:
subject_id = 1046

In [123]:
dataset.run_preprocessing(specific_key_dataset=subject_id, create_synthetic_samples=True, k_neighbors=5)

stage II Preprocessing: 100%|████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.53s/it]


In [124]:
dataset.run_test_preprocessing(specific_key_dataset=subject_id)

Preprocessing Test Data: 100%|███████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.19it/s]


## Train Validation Split

In [125]:
dataset.train_validation_split(val_proportion=0.2)

In [126]:
print('Training size:',len(dataset.train_list))
print('Validation size:',len(dataset.val_list))

Training size: 118
Validation size: 29


## Run Training/Validation

In [127]:
# Hyperparameters
hparams = Namespace(**{'learning_rate':learning_rate, 'train_batch_size': train_batch_size,
                       'val_batch_size': val_batch_size})

In [128]:
# Run training/validation of model
model = LightningEnsemble(dataset_name=dataset_name,hparams=hparams,train_list=dataset.train_list,
                       val_list=dataset.val_list,label_class=label_class,
                          path_to_null_preds=path_to_null_preds, gpus=gpus)
logger = TensorBoardLogger("lightning_logs", name=model_name, version=f'subject_{subject_id}_{label_class}')
trainer = pl.Trainer(early_stop_callback=early_stop_callback,
                     max_epochs=max_epochs, gpus=gpus, logger=logger,checkpoint_callback=False)
trainer.fit(model)

C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\gradient_accumulation_scheduler.py:43: DeprecationWarning: Epochs indexing of `scheduling` starts from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\trainer\training_loop.py:280: DeprecationWarning: Displayed epoch numbers in the progress bar start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


1

## Make Predictions

In [129]:
# Predict and fill in submission template
for spec, id_, subject, ts in dataset.test_data_list:
    output = model([torch.unsqueeze(TF.to_tensor(spec).cuda(), dim=0),
                    torch.unsqueeze(torch.tensor(ts).type('torch.FloatTensor').view(ts.shape[0], -1),dim=0),
                   subject])
    prediction = output.argmax(dim=1, keepdim=True).item()
    df_index = tst_sc3.index[tst_sc3['measurement_id'] == id_]
    tst_sc3.prediction[df_index] = prediction

LightningEnsemble.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import torch.nn as nn


In [130]:
 tst_sc3.subject_id.unique()

array([1004.0, 1006.0, 1007.0, 1019.0, 1020.0, 1023.0, 1032.0, 1034.0,
       1038.0, 1043.0, 1046.0, 1048.0, 1049.0, 'hbv013', 'hbv038',
       'hbv023', 'hbv054', 'hbv022', 'hbv012'], dtype=object)

# Subject 1048 Model

In [131]:
torch.cuda.empty_cache()

In [132]:
subject_id = 1048

In [133]:
dataset.run_preprocessing(specific_key_dataset=subject_id, create_synthetic_samples=True, k_neighbors=5)

stage II Preprocessing: 100%|████████████████████████████████████████████████████████████| 1/1 [00:05<00:00,  5.67s/it]


In [134]:
dataset.run_test_preprocessing(specific_key_dataset=subject_id)

Preprocessing Test Data: 100%|███████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.12s/it]


## Train Validation Split

In [135]:
dataset.train_validation_split(val_proportion=0.2)

In [136]:
print('Training size:',len(dataset.train_list))
print('Validation size:',len(dataset.val_list))

Training size: 192
Validation size: 47


## Run Training/Validation

In [137]:
# Hyperparameters
hparams = Namespace(**{'learning_rate':learning_rate, 'train_batch_size': train_batch_size,
                       'val_batch_size': val_batch_size})

In [138]:
# Run training/validation of model
model = LightningEnsemble(dataset_name=dataset_name,hparams=hparams,train_list=dataset.train_list,
                       val_list=dataset.val_list,label_class=label_class,
                          path_to_null_preds=path_to_null_preds, gpus=gpus)
logger = TensorBoardLogger("lightning_logs", name=model_name, version=f'subject_{subject_id}_{label_class}')
trainer = pl.Trainer(early_stop_callback=early_stop_callback,
                     max_epochs=max_epochs, gpus=gpus, logger=logger,checkpoint_callback=False)
trainer.fit(model)

C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\gradient_accumulation_scheduler.py:43: DeprecationWarning: Epochs indexing of `scheduling` starts from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\trainer\training_loop.py:280: DeprecationWarning: Displayed epoch numbers in the progress bar start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


1

## Make Predictions

In [139]:
# Predict and fill in submission template
for spec, id_, subject, ts in dataset.test_data_list:
    output = model([torch.unsqueeze(TF.to_tensor(spec).cuda(), dim=0),
                    torch.unsqueeze(torch.tensor(ts).type('torch.FloatTensor').view(ts.shape[0], -1),dim=0),
                   subject])
    prediction = output.argmax(dim=1, keepdim=True).item()
    df_index = tst_sc3.index[tst_sc3['measurement_id'] == id_]
    tst_sc3.prediction[df_index] = prediction

LightningEnsemble.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import torch.nn as nn


In [140]:
 tst_sc3.subject_id.unique()

array([1004.0, 1006.0, 1007.0, 1019.0, 1020.0, 1023.0, 1032.0, 1034.0,
       1038.0, 1043.0, 1046.0, 1048.0, 1049.0, 'hbv013', 'hbv038',
       'hbv023', 'hbv054', 'hbv022', 'hbv012'], dtype=object)

# Subject 1049 Model

In [141]:
torch.cuda.empty_cache()

In [142]:
subject_id = 1049

In [143]:
dataset.run_preprocessing(specific_key_dataset=subject_id, create_synthetic_samples=True, k_neighbors=5)

stage II Preprocessing: 100%|████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.96s/it]


In [144]:
dataset.run_test_preprocessing(specific_key_dataset=subject_id)

Preprocessing Test Data: 100%|███████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.08s/it]


## Train Validation Split

In [145]:
dataset.train_validation_split(val_proportion=0.2)

In [146]:
print('Training size:',len(dataset.train_list))
print('Validation size:',len(dataset.val_list))

Training size: 100
Validation size: 24


## Run Training/Validation

In [147]:
# Hyperparameters
hparams = Namespace(**{'learning_rate':learning_rate, 'train_batch_size': train_batch_size,
                       'val_batch_size': val_batch_size})

In [148]:
# Run training/validation of model
model = LightningEnsemble(dataset_name=dataset_name,hparams=hparams,train_list=dataset.train_list,
                       val_list=dataset.val_list,label_class=label_class,
                          path_to_null_preds=path_to_null_preds, gpus=gpus)
logger = TensorBoardLogger("lightning_logs", name=model_name, version=f'subject_{subject_id}_{label_class}')
trainer = pl.Trainer(early_stop_callback=early_stop_callback,
                     max_epochs=max_epochs, gpus=gpus, logger=logger,checkpoint_callback=False)
trainer.fit(model)

C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\gradient_accumulation_scheduler.py:43: DeprecationWarning: Epochs indexing of `scheduling` starts from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\trainer\training_loop.py:280: DeprecationWarning: Displayed epoch numbers in the progress bar start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


1

## Make Predictions

In [149]:
# Predict and fill in submission template
for spec, id_, subject, ts in dataset.test_data_list:
    output = model([torch.unsqueeze(TF.to_tensor(spec).cuda(), dim=0),
                    torch.unsqueeze(torch.tensor(ts).type('torch.FloatTensor').view(ts.shape[0], -1),dim=0),
                   subject])
    prediction = output.argmax(dim=1, keepdim=True).item()
    df_index = tst_sc3.index[tst_sc3['measurement_id'] == id_]
    tst_sc3.prediction[df_index] = prediction

LightningEnsemble.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import torch.nn as nn


In [150]:
tst_sc3.subject_id.unique()

array([1004.0, 1006.0, 1007.0, 1019.0, 1020.0, 1023.0, 1032.0, 1034.0,
       1038.0, 1043.0, 1046.0, 1048.0, 1049.0, 'hbv013', 'hbv038',
       'hbv023', 'hbv054', 'hbv022', 'hbv012'], dtype=object)

# Create REAL Dataset

In [151]:
############### REAL config ###############

dataset_name = 'REAL'
label_class = 'tremor'
sort_by = 'subject_id'
train_ts_dir = 'Data/BEATPDchallenge/BeatPDRealPDupdated/real-pd.training_data_updated/training_data/'
train_label_dir = 'Data/BEATPDchallenge/data_labels-realpd/REAL-PD_Training_Data_IDs_Labels.csv'
ancil_ts_dir = 'Data/BEATPDchallenge/BeatPDRealPDupdated/real-pd.ancillary_data_updated/ancillary_data/'
ancil_label_dir = 'Data/BEATPDchallenge/data_labels-realpd/REAL-PD_Ancillary_Data_IDs_Labels.csv'
test_ts_dir='Data/BEATPDchallenge/BeatPDRealPDupdated/real-pd.testing_data_updated/testing_data/'
test_data_id_dir='Data/BEATPDchallenge/REAL-PD_Test_Data_IDs.csv'

In [152]:
REALdataset = Dataset(dataset_name=dataset_name, sort_by=sort_by, label_class = label_class,
                           train_ts_dir=train_ts_dir, train_label_dir=train_label_dir,
                           ancil_ts_dir=ancil_ts_dir, ancil_label_dir=ancil_label_dir,
                           combine_ancil=False,test_ts_dir=test_ts_dir, test_data_id_dir=test_data_id_dir)

Creating Test Dictionary: 100%|██████████████████████████████████████████████████████| 191/191 [00:30<00:00,  6.20it/s]


Issue with 56 measurements.
 Enter self.issue_measuements for a list of them.


In [153]:
REALdataset.data_dict.keys()

dict_keys(['hbv013', 'hbv038', 'hbv017', 'hbv023', 'hbv051', 'hbv077', 'hbv054', 'hbv014', 'hbv018', 'hbv043', 'hbv022', 'hbv012'])

In [154]:
REALdataset.test_data_dict.keys()

dict_keys(['hbv013', 'hbv038', 'hbv017', 'hbv023', 'hbv051', 'hbv077', 'hbv054', 'hbv014', 'hbv018', 'hbv043', 'hbv022', 'hbv012'])

In [155]:
tst_sc3.subject_id.unique()

array([1004.0, 1006.0, 1007.0, 1019.0, 1020.0, 1023.0, 1032.0, 1034.0,
       1038.0, 1043.0, 1046.0, 1048.0, 1049.0, 'hbv013', 'hbv038',
       'hbv023', 'hbv054', 'hbv022', 'hbv012'], dtype=object)

# Subject hbv013 Model

In [156]:
torch.cuda.empty_cache()

In [157]:
subject_id = 'hbv013'

In [158]:
REALdataset.run_preprocessing(specific_key_dataset=subject_id, create_synthetic_samples=True, k_neighbors=5)

stage II Preprocessing: 100%|████████████████████████████████████████████████████████████| 1/1 [00:11<00:00, 11.49s/it]

73 samples with missing labels.
 Labels were replaced with key label mean.


In [159]:
REALdataset.run_test_preprocessing(specific_key_dataset=subject_id)

Preprocessing Test Data: 100%|███████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.93s/it]


## Train Validation Split

In [160]:
REALdataset.train_validation_split(val_proportion=0.2)

In [161]:
print('Training size:',len(REALdataset.train_list))
print('Validation size:',len(REALdataset.val_list))

Training size: 292
Validation size: 73


## Run Training/Validation

In [162]:
# Hyperparameters
hparams = Namespace(**{'learning_rate':learning_rate, 'train_batch_size': train_batch_size,
                       'val_batch_size': val_batch_size})

In [163]:
# Run training/validation of model
model = LightningEnsemble(dataset_name=dataset_name,hparams=hparams,train_list=REALdataset.train_list,
                       val_list=REALdataset.val_list,label_class=label_class,
                          path_to_null_preds=path_to_null_preds, gpus=gpus)
logger = TensorBoardLogger("lightning_logs", name=model_name, version=f'subject_{subject_id}_{label_class}')
trainer = pl.Trainer(early_stop_callback=early_stop_callback,
                     max_epochs=max_epochs, gpus=gpus, logger=logger,checkpoint_callback=False)
trainer.fit(model)

C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\gradient_accumulation_scheduler.py:43: DeprecationWarning: Epochs indexing of `scheduling` starts from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\trainer\training_loop.py:280: DeprecationWarning: Displayed epoch numbers in the progress bar start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\early_stopping.py:117: DeprecationWarning: Displayed epoch numbers by `EarlyStopping` start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


1

## Make Predictions

In [164]:
# Predict and fill in submission template
for spec, id_, subject, ts in REALdataset.test_data_list:
    output = model([torch.unsqueeze(TF.to_tensor(spec).cuda(), dim=0),
                    torch.unsqueeze(torch.tensor(ts).type('torch.FloatTensor').view(ts.shape[0], -1),dim=0),
                   subject])
    prediction = output.argmax(dim=1, keepdim=True).item()
    df_index = tst_sc3.index[tst_sc3['measurement_id'] == id_]
    tst_sc3.prediction[df_index] = prediction

LightningEnsemble.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import torch.nn as nn


In [165]:
tst_sc3.subject_id.unique()

array([1004.0, 1006.0, 1007.0, 1019.0, 1020.0, 1023.0, 1032.0, 1034.0,
       1038.0, 1043.0, 1046.0, 1048.0, 1049.0, 'hbv013', 'hbv038',
       'hbv023', 'hbv054', 'hbv022', 'hbv012'], dtype=object)

# Subject hbv038 Model

In [166]:
subject_id = 'hbv038'

In [167]:
REALdataset.run_preprocessing(specific_key_dataset=subject_id, create_synthetic_samples=True, k_neighbors=5)

stage II Preprocessing: 100%|████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.86s/it]

27 samples with missing labels.
 Labels were replaced with key label mean.


In [168]:
REALdataset.run_test_preprocessing(specific_key_dataset=subject_id)

Preprocessing Test Data: 100%|███████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.47it/s]


## Train Validation Split

In [169]:
REALdataset.train_validation_split(val_proportion=0.2)

In [170]:
print('Training size:',len(REALdataset.train_list))
print('Validation size:',len(REALdataset.val_list))

Training size: 87
Validation size: 21


## Run Training/Validation

In [171]:
# Hyperparameters
hparams = Namespace(**{'learning_rate':learning_rate, 'train_batch_size': train_batch_size,
                       'val_batch_size': val_batch_size})

In [172]:
# Run training/validation of model
model = LightningEnsemble(dataset_name=dataset_name,hparams=hparams,train_list=REALdataset.train_list,
                       val_list=REALdataset.val_list,label_class=label_class,
                          path_to_null_preds=path_to_null_preds, gpus=gpus)
logger = TensorBoardLogger("lightning_logs", name=model_name, version=f'subject_{subject_id}_{label_class}')
trainer = pl.Trainer(early_stop_callback=early_stop_callback,
                     max_epochs=max_epochs, gpus=gpus, logger=logger,checkpoint_callback=False)
trainer.fit(model)

C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\gradient_accumulation_scheduler.py:43: DeprecationWarning: Epochs indexing of `scheduling` starts from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\trainer\training_loop.py:280: DeprecationWarning: Displayed epoch numbers in the progress bar start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\early_stopping.py:117: DeprecationWarning: Displayed epoch numbers by `EarlyStopping` start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


1

## Make Predictions

In [173]:
# Predict and fill in submission template
for spec, id_, subject, ts in REALdataset.test_data_list:
    output = model([torch.unsqueeze(TF.to_tensor(spec).cuda(), dim=0),
                    torch.unsqueeze(torch.tensor(ts).type('torch.FloatTensor').view(ts.shape[0], -1),dim=0),
                   subject])
    prediction = output.argmax(dim=1, keepdim=True).item()
    df_index = tst_sc3.index[tst_sc3['measurement_id'] == id_]
    tst_sc3.prediction[df_index] = prediction

LightningEnsemble.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import torch.nn as nn


In [174]:
tst_sc3.subject_id.unique()

array([1004.0, 1006.0, 1007.0, 1019.0, 1020.0, 1023.0, 1032.0, 1034.0,
       1038.0, 1043.0, 1046.0, 1048.0, 1049.0, 'hbv013', 'hbv038',
       'hbv023', 'hbv054', 'hbv022', 'hbv012'], dtype=object)

# Subject hbv023 Model

In [175]:
torch.cuda.empty_cache()

In [176]:
subject_id = 'hbv023'

In [177]:
REALdataset.run_preprocessing(specific_key_dataset=subject_id, create_synthetic_samples=True, k_neighbors=5)

stage II Preprocessing: 100%|████████████████████████████████████████████████████████████| 1/1 [00:05<00:00,  5.19s/it]


In [178]:
REALdataset.run_test_preprocessing(specific_key_dataset=subject_id)

Preprocessing Test Data: 100%|███████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.46it/s]


## Train Validation Split

In [179]:
REALdataset.train_validation_split(val_proportion=0.2)

In [180]:
print('Training size:',len(REALdataset.train_list))
print('Validation size:',len(REALdataset.val_list))

Training size: 164
Validation size: 40


## Run Training/Validation

In [181]:
# Hyperparameters
hparams = Namespace(**{'learning_rate':learning_rate, 'train_batch_size': train_batch_size,
                       'val_batch_size': val_batch_size})

In [182]:
# Run training/validation of model
model = LightningEnsemble(dataset_name=dataset_name,hparams=hparams,train_list=REALdataset.train_list,
                       val_list=REALdataset.val_list,label_class=label_class,
                          path_to_null_preds=path_to_null_preds, gpus=gpus)
logger = TensorBoardLogger("lightning_logs", name=model_name, version=f'subject_{subject_id}_{label_class}')
trainer = pl.Trainer(early_stop_callback=early_stop_callback,
                     max_epochs=max_epochs, gpus=gpus, logger=logger,checkpoint_callback=False)
trainer.fit(model)

C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\gradient_accumulation_scheduler.py:43: DeprecationWarning: Epochs indexing of `scheduling` starts from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\trainer\training_loop.py:280: DeprecationWarning: Displayed epoch numbers in the progress bar start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\early_stopping.py:117: DeprecationWarning: Displayed epoch numbers by `EarlyStopping` start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


1

## Make Predictions

In [183]:
# Predict and fill in submission template
for spec, id_, subject, ts in REALdataset.test_data_list:
    output = model([torch.unsqueeze(TF.to_tensor(spec).cuda(), dim=0),
                    torch.unsqueeze(torch.tensor(ts).type('torch.FloatTensor').view(ts.shape[0], -1),dim=0),
                   subject])
    prediction = output.argmax(dim=1, keepdim=True).item()
    df_index = tst_sc3.index[tst_sc3['measurement_id'] == id_]
    tst_sc3.prediction[df_index] = prediction

LightningEnsemble.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import torch.nn as nn


In [184]:
tst_sc3.subject_id.unique()

array([1004.0, 1006.0, 1007.0, 1019.0, 1020.0, 1023.0, 1032.0, 1034.0,
       1038.0, 1043.0, 1046.0, 1048.0, 1049.0, 'hbv013', 'hbv038',
       'hbv023', 'hbv054', 'hbv022', 'hbv012'], dtype=object)

# Subject hbv054 Model

In [185]:
torch.cuda.empty_cache()

In [186]:
subject_id = 'hbv054'

In [187]:
REALdataset.run_preprocessing(specific_key_dataset=subject_id, create_synthetic_samples=True, k_neighbors=5)

stage II Preprocessing: 100%|████████████████████████████████████████████████████████████| 1/1 [00:07<00:00,  7.77s/it]

96 samples with missing labels.
 Labels were replaced with key label mean.


In [188]:
REALdataset.run_test_preprocessing(specific_key_dataset=subject_id)

Preprocessing Test Data: 100%|███████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.00it/s]


## Train Validation Split

In [189]:
REALdataset.train_validation_split(val_proportion=0.2)

In [190]:
print('Training size:',len(REALdataset.train_list))
print('Validation size:',len(REALdataset.val_list))

Training size: 231
Validation size: 57


## Run Training/Validation

In [191]:
# Hyperparameters
hparams = Namespace(**{'learning_rate':learning_rate, 'train_batch_size': train_batch_size,
                       'val_batch_size': val_batch_size})

In [192]:
# Run training/validation of model
model = LightningEnsemble(dataset_name=dataset_name,hparams=hparams,train_list=REALdataset.train_list,
                       val_list=REALdataset.val_list,label_class=label_class,
                          path_to_null_preds=path_to_null_preds, gpus=gpus)
logger = TensorBoardLogger("lightning_logs", name=model_name, version=f'subject_{subject_id}_{label_class}')
trainer = pl.Trainer(early_stop_callback=early_stop_callback,
                     max_epochs=max_epochs, gpus=gpus, logger=logger,checkpoint_callback=False)
trainer.fit(model)

C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\gradient_accumulation_scheduler.py:43: DeprecationWarning: Epochs indexing of `scheduling` starts from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\trainer\training_loop.py:280: DeprecationWarning: Displayed epoch numbers in the progress bar start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


1

## Make Predictions

In [193]:
# Predict and fill in submission template
for spec, id_, subject, ts in REALdataset.test_data_list:
    output = model([torch.unsqueeze(TF.to_tensor(spec).cuda(), dim=0),
                    torch.unsqueeze(torch.tensor(ts).type('torch.FloatTensor').view(ts.shape[0], -1),dim=0),
                   subject])
    prediction = output.argmax(dim=1, keepdim=True).item()
    df_index = tst_sc3.index[tst_sc3['measurement_id'] == id_]
    tst_sc3.prediction[df_index] = prediction

LightningEnsemble.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import torch.nn as nn


In [194]:
tst_sc3.subject_id.unique()

array([1004.0, 1006.0, 1007.0, 1019.0, 1020.0, 1023.0, 1032.0, 1034.0,
       1038.0, 1043.0, 1046.0, 1048.0, 1049.0, 'hbv013', 'hbv038',
       'hbv023', 'hbv054', 'hbv022', 'hbv012'], dtype=object)

# Subject hbv022 Model

In [195]:
torch.cuda.empty_cache()

In [196]:
subject_id = 'hbv022'

In [197]:
REALdataset.run_preprocessing(specific_key_dataset=subject_id, create_synthetic_samples=True, k_neighbors=5)

stage II Preprocessing: 100%|████████████████████████████████████████████████████████████| 1/1 [00:09<00:00,  9.11s/it]

85 samples with missing labels.
 Labels were replaced with key label mean.


In [198]:
REALdataset.run_test_preprocessing(specific_key_dataset=subject_id)

Preprocessing Test Data: 100%|███████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.27s/it]


## Train Validation Split

In [199]:
REALdataset.train_validation_split(val_proportion=0.2)

In [200]:
print('Training size:',len(REALdataset.train_list))
print('Validation size:',len(REALdataset.val_list))

Training size: 272
Validation size: 68


## Run Training/Validation

In [201]:
# Hyperparameters
hparams = Namespace(**{'learning_rate':learning_rate, 'train_batch_size': train_batch_size,
                       'val_batch_size': val_batch_size})

In [202]:
# Run training/validation of model
model = LightningEnsemble(dataset_name=dataset_name,hparams=hparams,train_list=REALdataset.train_list,
                       val_list=REALdataset.val_list,label_class=label_class,
                          path_to_null_preds=path_to_null_preds, gpus=gpus)
logger = TensorBoardLogger("lightning_logs", name=model_name, version=f'subject_{subject_id}_{label_class}')
trainer = pl.Trainer(early_stop_callback=early_stop_callback,
                     max_epochs=max_epochs, gpus=gpus, logger=logger,checkpoint_callback=False)
trainer.fit(model)

C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\gradient_accumulation_scheduler.py:43: DeprecationWarning: Epochs indexing of `scheduling` starts from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\trainer\training_loop.py:280: DeprecationWarning: Displayed epoch numbers in the progress bar start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\early_stopping.py:117: DeprecationWarning: Displayed epoch numbers by `EarlyStopping` start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


1

## Make Predictions

In [203]:
# Predict and fill in submission template
for spec, id_, subject, ts in REALdataset.test_data_list:
    output = model([torch.unsqueeze(TF.to_tensor(spec).cuda(), dim=0),
                    torch.unsqueeze(torch.tensor(ts).type('torch.FloatTensor').view(ts.shape[0], -1),dim=0),
                   subject])
    prediction = output.argmax(dim=1, keepdim=True).item()
    df_index = tst_sc3.index[tst_sc3['measurement_id'] == id_]
    tst_sc3.prediction[df_index] = prediction

LightningEnsemble.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import torch.nn as nn


In [204]:
tst_sc3.subject_id.unique()

array([1004.0, 1006.0, 1007.0, 1019.0, 1020.0, 1023.0, 1032.0, 1034.0,
       1038.0, 1043.0, 1046.0, 1048.0, 1049.0, 'hbv013', 'hbv038',
       'hbv023', 'hbv054', 'hbv022', 'hbv012'], dtype=object)

# Subject hbv012 Model

In [205]:
torch.cuda.empty_cache()

In [206]:
subject_id = 'hbv012'

In [207]:
REALdataset.run_preprocessing(specific_key_dataset=subject_id, create_synthetic_samples=True, k_neighbors=5)

stage II Preprocessing: 100%|████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.71s/it]


In [208]:
REALdataset.run_test_preprocessing(specific_key_dataset=subject_id)

Preprocessing Test Data: 100%|███████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.60it/s]


## Train Validation Split

In [209]:
REALdataset.train_validation_split(val_proportion=0.2)

In [210]:
print('Training size:',len(REALdataset.train_list))
print('Validation size:',len(REALdataset.val_list))

Training size: 108
Validation size: 27


## Run Training/Validation

In [211]:
# Hyperparameters
hparams = Namespace(**{'learning_rate':learning_rate, 'train_batch_size': train_batch_size,
                       'val_batch_size': val_batch_size})

In [212]:
# Run training/validation of model
model = LightningEnsemble(dataset_name=dataset_name,hparams=hparams,train_list=REALdataset.train_list,
                       val_list=REALdataset.val_list,label_class=label_class,
                          path_to_null_preds=path_to_null_preds, gpus=gpus)
logger = TensorBoardLogger("lightning_logs", name=model_name, version=f'subject_{subject_id}_{label_class}')
trainer = pl.Trainer(early_stop_callback=early_stop_callback,
                     max_epochs=max_epochs, gpus=gpus, logger=logger,checkpoint_callback=False)
trainer.fit(model)

C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\callbacks\gradient_accumulation_scheduler.py:43: DeprecationWarning: Epochs indexing of `scheduling` starts from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


C:\Users\Jonathan\Anaconda3\envs\researchenv\lib\site-packages\pytorch_lightning\trainer\training_loop.py:280: DeprecationWarning: Displayed epoch numbers in the progress bar start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  ' but will start from "0" in v0.8.0.', DeprecationWarning)


1

## Make Predictions

In [213]:
# Predict and fill in submission template
for spec, id_, subject, ts in REALdataset.test_data_list:
    output = model([torch.unsqueeze(TF.to_tensor(spec).cuda(), dim=0),
                    torch.unsqueeze(torch.tensor(ts).type('torch.FloatTensor').view(ts.shape[0], -1),dim=0),
                   subject])
    prediction = output.argmax(dim=1, keepdim=True).item()
    df_index = tst_sc3.index[tst_sc3['measurement_id'] == id_]
    tst_sc3.prediction[df_index] = prediction

LightningEnsemble.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import torch.nn as nn


In [214]:
tst_sc3.subject_id.unique()

array([1004.0, 1006.0, 1007.0, 1019.0, 1020.0, 1023.0, 1032.0, 1034.0,
       1038.0, 1043.0, 1046.0, 1048.0, 1049.0, 'hbv013', 'hbv038',
       'hbv023', 'hbv054', 'hbv022', 'hbv012'], dtype=object)

# Save Predictions

In [215]:
del tst_sc3['subject_id']

In [216]:
tst_sc3.prediction = tst_sc3.prediction.astype(int)

In [219]:
#Save submission df to CSV
tst_sc3.to_csv('Data/tremor_predictions_final.csv', index=False)

In [220]:
# Check to see if csv saved correctly
pd.read_csv('Data/tremor_predictions_final.csv').head()

,measurement_id,prediction
0,0e80673b-8c90-474f-8388-00fdf25795e4,4
1,a7b61d40-9f26-4878-a2d7-bd766b8ae6f5,2
2,fc851afd-133c-4e8f-8deb-0079cc442358,2
3,90c2c320-1007-47b4-9f8e-cdd37067dfb2,2
4,379fd23b-e38f-4214-8a70-7a829004359f,0
